In [1]:
import os
import time
import dill
from typing import List
from functools import reduce
from calendar import monthrange

In [2]:
import json
import csv
import requests
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import urllib.request as urllib2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from random import sample, randint, shuffle
from sklearn.decomposition import PCA

In [4]:
import nltk
import text_summarizer
from gensim.utils import simple_preprocess
from gensim.summarization.summarizer import summarize
from gensim.models import Word2Vec, LsiModel
from gensim.test.utils import common_dictionary, common_corpus

In [5]:
from time import process_time

In [6]:
t = process_time()
dill.load_session('thesis_env2.db')
process_time() - t

-2866.8014740000003

In [7]:
len(politics), len(movies), len(books), len(sports)

(7370, 7370, 7370, 7370)

In [9]:
len(all_)

1685787

In [10]:
# mpol, mmov, mbooks, msports
# summaries_{pol, mov, bks, spts}
# m{pol, mov, bks, spts}_{s, b, c}
# bow_{pol, mov, bks, spts}
# ctrls_{pol, mov, bks, spts}

In [11]:
len(pairs), len(proflist), len(gender_specific)

(52, 205, 218)

In [12]:
vocab = set(mpol.wv.vocab)
for m in [mpol, mmov, mbooks, msports]:
    vocab &= set(m.wv.vocab)
len(vocab)

16581

In [15]:
vocab_all = set(vocab)
for m in [mpol_s, mbks_s, mmov_s, mspts_s] + \
[mpol_b, mbks_b, mmov_b, mspts_b] + \
[mpol_c, mbks_c, mmov_c, mspts_c]:
    vocab_all &= set(m.wv.vocab)
len(vocab_all)

5573

In [16]:
vocab2 = vocab_all

In [17]:
len(vocab), len(vocab2)

(16581, 5573)

In [13]:
with open('grammar.csv') as csv_file:
    grammar = csv_file.read().split(",")

In [14]:
len(adjectives), len(grammar), len(cities)

(393, 71, 2010)

# WEAT Bias

In [81]:
sciences = ["science", "technology", "physics", "chemistry", \
            "einstein", "nasa", "experiment", "astronomy"]

In [82]:
maths = ["math", "algebra", "geometry", "calculus", "equations", \
         "computation", "numbers", "addition"]

In [83]:
arts = ["poetry", "art", "dance", "literature", "novel", \
        "symphony", "drama", "sculpture"]

In [85]:
family = ["home", "parents", "children", "family", "cousins", \
          "marriage", "wedding", "relatives"]

In [187]:
female = ["female", "woman", "girl", "girls", "sister", "she", "her", \
          "daughter", "mother"]

In [190]:
male = ["male", "man", "boy", "boys", "brother", "he", "him", "son", "father"]

In [88]:
career = list(filter(lambda x: x in vocab, career))
family = list(filter(lambda x: x in vocab, family))
arts = list(filter(lambda x: x in vocab, arts))
maths = list(filter(lambda x: x in vocab, maths))
sciences = list(filter(lambda x: x in vocab, sciences))

In [145]:
female = list(filter(lambda x: x in vocab2, female))
male = list(filter(lambda x: x in vocab2, male))

In [32]:
def cos_sim(m, x, y):
    wv1 = m.wv.get_vector(x)
    wv2 = m.wv.get_vector(y)
    sim = np.dot(wv1, wv2)/(np.linalg.norm(wv1)*np.linalg.norm(wv2))
    return sim

In [42]:
def s_word(m, w, A, B):
    return np.mean([cos_sim(m, w, a) for a in A]) - np.mean([cos_sim(m, w, b) for b in B])


In [58]:
def s_set(m, X, Y, A, B):
    return sum([s_word(m, x, A, B) for x in X]) - sum([s_word(m, y, A, B) for y in Y])

In [93]:
def effect_size(m, X, Y, A, B):
    num = np.mean([s_word(m, x, A, B) for x in X]) - np.mean([s_word(m, y, A, B) for y in Y])
    return num/np.std([s_word(m, x, A, B) for x in X + Y])

In [95]:
def mdictadd(models):
    for i in range(len(models)):
        mdict[list(mdict)[i]].append(models[i])

In [126]:
combined = politics+books+movies+sports
len(combined)

29480

In [129]:
# train word embeddings
mcombined = getw2v(combined)

In [130]:
profs = set(proflist) & vocab2

In [131]:
gcombined = getg(mcombined)

0.43000886


In [132]:
def gproj(g, m, w1):
    wvec = m.wv.get_vector(w1)
    p = np.dot(g, wvec)/(np.linalg.norm(g)*np.linalg.norm(wvec))
    return p

In [152]:
for p in profs:
    b = gproj(gcombined, mcombined, p)
    if b > .15 or b < -.01:
        print(p, b)

officer 0.19345711
teacher -0.18449657
judge -0.07651596
worker -0.13081126
character -0.08281248
coach 0.34330636
doctor -0.1050056
author -0.039736334
analyst 0.15470724
commissioner 0.15923788
substitute -0.03780298
prosecutor -0.013197501
surgeon -0.023146214
editor -0.050578646
president 0.1799249
manager 0.15750493
writer -0.03770707
artist -0.046602905
student -0.16729335
dean 0.29737547
photographer -0.13515998
singer -0.0139438845
captain 0.27266547
attorney -0.05113222
director 0.17244485
teenager -0.23446582


In [153]:
stereotypes_m = ['captain', 'president', 'professor', 'dean', \
                 'commissioner', 'coach', 'director', 'officer', \
                 'manager']
stereotypes_f = ['teacher', 'author', 'secretary', 'writer', \
                 'artist', 'singer', 'teenager', 'editor', \
                 'photographer']

In [124]:
mdict = {"politics": [], "books": [], "movies": [], "sports": []}
mdictadd([mpol, mbooks, mmov, msports])
mdictadd([mpol_s, mbks_s, mmov_s, mspts_s])
mdictadd([mpol_b, mbks_b, mmov_b, mspts_b])
mdictadd([mpol_c, mbks_c, mmov_c, mspts_c])

In [143]:
def bias(m):
    return effect_size(m, male, female, stereotypes_m, stereotypes_f)

In [191]:
print(labels[0])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias(mdict[i][0]))
print(labels[1])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias(mdict[i][1]))
print(labels[2])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias(mdict[i][2]))
print(labels[3])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias(mdict[i][3]))

original
  politics 0.447
     books 1.075
    movies 1.201
    sports 0.913
summary (TextRank)
  politics 0.399
     books 0.765
    movies 0.257
    sports 1.035
summary (Centroid-BOW)
  politics 0.159
     books 0.701
    movies 0.368
    sports 0.838
summary (control)
  politics 0.212
     books 0.395
    movies 0.305
    sports 0.678


In [179]:
def bias_c(m, a, b):
    return effect_size(m, male, female, a, b)

In [165]:
male

['male', 'man', 'boy', 'brother', 'he', 'him', 'son', 'father']

In [166]:
female

['female', 'woman', 'girl', 'sister', 'she', 'her', 'daughter', 'mother']

In [175]:
male2 = ['husband', 'brothers', 'king', \
         'grandfather', 'guy', 'spokesman', 'uncle', \
         'himself', 'men']

In [176]:
female2 = ['wife', 'sisters', 'queen', ''\
           'grandmother', 'lady', 'spokeswoman', 'widow', \
           'herself', 'women']

In [180]:
print(labels[0])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_c(mdict[i][0], male2, female2))
print(labels[1])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_c(mdict[i][1], male2, female2))
print(labels[2])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_c(mdict[i][2], male2, female2))
print(labels[3])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_c(mdict[i][3], male2, female2))

original
  politics 1.496
     books 1.675
    movies 1.730
    sports 1.348
summary (TextRank)
  politics 0.613
     books 1.093
    movies 1.086
    sports 0.907
summary (Centroid-BOW)
  politics 0.557
     books 1.392
    movies 1.169
    sports 1.037
summary (control)
  politics 0.691
     books 1.061
    movies 0.949
    sports 0.544


In [185]:
def bias_r(m, a, b):
    tot = a + b
    shuffle(tot)
    return effect_size(m, male, female, tot[:len(a)], tot[len(a):])

In [194]:
print(labels[0])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_r(mdict[i][0], male2, female2))
print(labels[1])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_r(mdict[i][1], male2, female2))
print(labels[2])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_r(mdict[i][2], male2, female2))
print(labels[3])
for i in mdict:
    print("%10s" % i, \
          "%.3f" % bias_r(mdict[i][3], male2, female2))

original
  politics -0.199
     books -0.321
    movies 0.871
    sports -0.960
summary (TextRank)
  politics 0.273
     books 0.842
    movies -0.393
    sports -0.559
summary (Centroid-BOW)
  politics 0.129
     books 0.119
    movies -0.032
    sports 0.719
summary (control)
  politics -0.572
     books 0.199
    movies -0.030
    sports -0.748


In [21]:
def gdictadd(models):
    for i in range(len(models)):
        gdict[list(gdict)[i]].append(getg(models[i]))

In [22]:
labels = ["original", "summary (TextRank)", "summary (Centroid-BOW)", "summary (control)"]
gdict = {"politics": [], "books": [], "movies": [], "sports": []}
gdictadd([mpol, mmov, mbooks, msports])
gdictadd([mpol_s, mbks_s, mmov_s, mspts_s])
gdictadd([mpol_b, mbks_b, mmov_b, mspts_b])
gdictadd([mpol_c, mbks_c, mmov_c, mspts_c])

0.34343678
0.52675015
0.5915069
0.35275307
0.31522945
0.78374213
0.62817353
0.49360958
0.31965494
0.8359838
0.7387123
0.51820844
0.32400513
0.768514
0.6437023
0.48822078


In [62]:
def gproj(g, m, w1):
    wvec = m.wv.get_vector(w1)
    p = np.dot(g, wvec)/(np.linalg.norm(g)*np.linalg.norm(wvec))
    return p

In [63]:
def bias(g, m, vocab, wordlist):
    b = 0
    l = list(filter(lambda x: x in vocab, wordlist))
    for x in l:
        b += abs(gproj(g, m, x))
    return b/len(l)

In [24]:
print(labels[0])
for i in gdict:
    print("%10s" % i, \
          "%.3f" % bias(gdict[i][0], mdict[i][0], vocab, proflist), \
          "%.3f" % bias(gdict[i][0], mdict[i][0], vocab, adjectives), \
          "%.3f" % bias(gdict[i][0], mdict[i][0], vocab, cities), \
          "%.3f" % bias(gdict[i][0], mdict[i][0], vocab, objects), \
          "%.3f" % bias(gdict[i][0], mdict[i][0], vocab, gender_specific))

original
  politics 0.099 0.066 0.072 0.058 0.120
     books 0.086 0.092 0.089 0.080 0.173
    movies 0.096 0.069 0.077 0.065 0.155
    sports 0.327 0.175 0.258 0.228 0.372


In [25]:
print(labels[1])
for i in gdict:
    print("%10s" % i, \
          "%.3f" % bias(gdict[i][1], mdict[i][1], vocab2, proflist), \
          "%.3f" % bias(gdict[i][1], mdict[i][1], vocab2, adjectives), \
          "%.3f" % bias(gdict[i][1], mdict[i][1], vocab2, cities), \
          "%.3f" % bias(gdict[i][1], mdict[i][1], vocab2, objects), \
          "%.3f" % bias(gdict[i][1], mdict[i][1], vocab2, gender_specific))

summary (TextRank)
  politics 0.392 0.111 0.294 0.146 0.269
     books 0.052 0.051 0.044 0.052 0.161
    movies 0.083 0.089 0.064 0.084 0.192
    sports 0.303 0.154 0.182 0.180 0.283


In [26]:
print(labels[2])
for i in gdict:
    print("%10s" % i, \
          "%.3f" % bias(gdict[i][2], mdict[i][2], vocab2, proflist), \
          "%.3f" % bias(gdict[i][2], mdict[i][2], vocab2, adjectives), \
          "%.3f" % bias(gdict[i][2], mdict[i][2], vocab2, cities), \
          "%.3f" % bias(gdict[i][2], mdict[i][2], vocab2, objects), \
          "%.3f" % bias(gdict[i][2], mdict[i][2], vocab2, gender_specific))

summary (Centroid-BOW)
  politics 0.512 0.152 0.446 0.229 0.405
     books 0.048 0.054 0.046 0.048 0.163
    movies 0.069 0.077 0.067 0.088 0.194
    sports 0.337 0.181 0.209 0.198 0.298


In [27]:
print(labels[3])
for i in gdict:
    print("%10s" % i, \
          "%.3f" % bias(gdict[i][3], mdict[i][3], vocab2, proflist), \
          "%.3f" % bias(gdict[i][3], mdict[i][3], vocab2, adjectives), \
          "%.3f" % bias(gdict[i][3], mdict[i][3], vocab2, cities), \
          "%.3f" % bias(gdict[i][3], mdict[i][3], vocab2, objects), \
          "%.3f" % bias(gdict[i][3], mdict[i][3], vocab2, gender_specific))

summary (control)
  politics 0.183 0.091 0.143 0.090 0.123
     books 0.046 0.045 0.037 0.049 0.137
    movies 0.066 0.069 0.051 0.068 0.172
    sports 0.246 0.115 0.145 0.135 0.197


In [28]:
gpol = getg(mpol)
res = sorted([(x, gproj(gpol, mpol, x)) for x in filter(lambda x: x in vocab, vocab)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gbooks = getg(mbooks)
res = sorted([(x, gproj(gbooks, mbooks, x)) for x in filter(lambda x: x in vocab, vocab)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gmov = getg(mmov)
res = sorted([(x, gproj(gmov, mmov, x)) for x in filter(lambda x: x in vocab, vocab)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gsports = getg(msports)
res = sorted([(x, gproj(gsports, msports, x)) for x in filter(lambda x: x in vocab, vocab)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")

0.34343678
her        -0.50 		 mr         0.43
women      -0.47 		 ouster     0.39
ms         -0.46 		 vladimir   0.37
she        -0.46 		 himself    0.36
feeding    -0.37 		 leader     0.35
disabled   -0.35 		 successor  0.35
herself    -0.35 		 loyalists  0.35
mental     -0.34 		 protégé    0.34
female     -0.34 		 predecessor 0.33
classroom  -0.32 		 hugo       0.33
0.5915069
mr         -0.45 		 her        0.60
baseball   -0.40 		 ms         0.58
himself    -0.40 		 herself    0.57
james      -0.37 		 she        0.51
military   -0.36 		 husband    0.47
john       -0.36 		 girl       0.43
ford       -0.36 		 mary       0.40
game       -0.35 		 women      0.40
al         -0.35 		 baby       0.40
henry      -0.34 		 sex        0.40
0.52675015
steven     -0.52 		 ms         0.63
george     -0.48 		 her        0.52
mr         -0.47 		 marie      0.52
lucas      -0.42 		 nicole     0.51
himself    -0.40 		 herself    0.51
franchise  -0.38 		 actress    0.47
bush       -0.38 		 queen      

In [35]:
gpol_s = getg(mpol_s)
res = sorted([(x, gproj(gpol_s, mpol_s, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gbks_s = getg(mbks_s)
res = sorted([(x, gproj(gbks_s, mbks_s, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gmov_s = getg(mmov_s)
res = sorted([(x, gproj(gmov_s, mmov_s, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gspts_s = getg(mspts_s)
res = sorted([(x, gproj(gspts_s, mspts_s, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")

0.31522945
these      -0.52 		 chairman   0.80
large      -0.44 		 secretary  0.78
low        -0.43 		 chief      0.77
without    -0.42 		 deputy     0.75
their      -0.40 		 leader     0.73
different  -0.40 		 paul       0.67
small      -0.39 		 president  0.67
making     -0.36 		 king       0.66
pay        -0.36 		 director   0.66
such       -0.36 		 general    0.66
0.78374213
mr         -0.38 		 her        0.63
he         -0.32 		 she        0.47
his        -0.30 		 herself    0.46
administration -0.28 		 husband    0.42
iraq       -0.27 		 baby       0.37
president  -0.26 		 girl       0.35
military   -0.26 		 children   0.31
bush       -0.26 		 mother     0.30
nations    -0.26 		 sister     0.30
history    -0.24 		 love       0.29
0.62817353
her        -0.57 		 disney     0.41
herself    -0.47 		 pictures   0.38
she        -0.44 		 production 0.35
husband    -0.41 		 studio     0.34
mother     -0.39 		 mr         0.34
ms         -0.38 		 producers  0.34
girl       -0.38 		 enterta

In [36]:
gpol_b = getg(mpol_b)
res = sorted([(x, gproj(gpol_b, mpol_b, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gbks_b = getg(mbks_b)
res = sorted([(x, gproj(gbks_b, mbks_b, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gmov_b = getg(mmov_b)
res = sorted([(x, gproj(gmov_b, mmov_b, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gspts_b = getg(mspts_b)
res = sorted([(x, gproj(gspts_b, mspts_b, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")

0.31965494
these      -0.48 		 chairman   0.92
without    -0.41 		 democrat   0.87
their      -0.41 		 james      0.84
prices     -0.38 		 lawyer     0.83
real       -0.36 		 representative 0.83
weapons    -0.35 		 jr         0.82
large      -0.35 		 peter      0.81
oil        -0.35 		 secretary  0.81
further    -0.33 		 john       0.81
enough     -0.33 		 joseph     0.80
0.8359838
her        -0.63 		 mr         0.44
she        -0.50 		 his        0.41
husband    -0.44 		 he         0.37
herself    -0.41 		 bush       0.33
girl       -0.30 		 administration 0.31
woman      -0.29 		 himself    0.29
ms         -0.29 		 president  0.24
baby       -0.28 		 nations    0.23
children   -0.28 		 states     0.23
mother     -0.27 		 clinton    0.21
0.7387123
her        -0.60 		 mr         0.43
she        -0.51 		 disney     0.34
herself    -0.48 		 his        0.32
girl       -0.44 		 studio     0.28
husband    -0.41 		 rights     0.28
woman      -0.41 		 company    0.28
mother     -0.35 		 major

In [37]:
gpol_c = getg(mpol_c)
res = sorted([(x, gproj(gpol_c, mpol_c, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gbks_c = getg(mbks_c)
res = sorted([(x, gproj(gbks_c, mbks_c, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gmov_c = getg(mmov_c)
res = sorted([(x, gproj(gmov_c, mmov_c, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")
gspts_c = getg(mspts_c)
res = sorted([(x, gproj(gspts_c, mspts_c, x)) for x in filter(lambda x: x in vocab2, vocab2)], key=lambda x: x[1])
for i in range(10):
    print(f"{res[i][0].ljust(10)} {res[i][1]:.2f} \t\t {res[-i-1][0].ljust(10)} {res[-i-1][1]:.2f}")

0.32400513
mail       -0.37 		 head       0.52
or         -0.35 		 chairman   0.51
high       -0.35 		 union      0.51
got        -0.33 		 president  0.50
few        -0.32 		 leadership 0.48
com        -0.32 		 secretary  0.48
young      -0.31 		 chief      0.47
children   -0.31 		 powers     0.47
isn        -0.29 		 coalition  0.47
these      -0.28 		 leader     0.47
0.768514
her        -0.61 		 mr         0.38
herself    -0.46 		 his        0.31
she        -0.46 		 war        0.29
husband    -0.41 		 he         0.28
children   -0.30 		 bush       0.27
love       -0.28 		 civil      0.24
ms         -0.27 		 administration 0.23
mother     -0.27 		 ii         0.23
girl       -0.27 		 iraq       0.22
woman      -0.27 		 policy     0.20
0.6437023
disney     -0.38 		 her        0.55
mr         -0.37 		 she        0.47
major      -0.34 		 herself    0.41
studio     -0.34 		 husband    0.41
company    -0.33 		 girl       0.40
entertainment -0.33 		 woman      0.35
distribution -0.33 		 mothe

## Plot PCA

## Regression for bias testing

In [30]:
def regress(g1, m1, g2, m2, wordlist):
    x = np.array([gproj(g1, m1, w) for w in wordlist if w in m1.wv.vocab and w in m2.wv.vocab])
    y = np.array([gproj(g2, m2, w) for w in wordlist if w in m1.wv.vocab and w in m2.wv.vocab])
    slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
    return slope, p_value

In [32]:
regress(gpol, mpol, getg(mpol_s), mpol_s, proflist)

0.31522945


(0.7564479357792341, 1.1464826034410933e-07)